## Sessionization in SQL and Python

Sessionization boils down to combining discrete events into sessions, a unit of measurement widely used when dealing with time series data.

Using data events, we need to define a threshold in time units to compare and classify differences in time from one event to the previous one. 

In this example, we have a dataset with user_id and mytimestamp, which represents the time the user has an interaction with the website. We choose to define a new session after 30min of inactivity. 

The goal is to detect on which line there's a new session in the ordered dataset. 

# SQL

Using PostgreSQL.

In [ ]:
# Calculate the interval between 2 lines of the same user.
SELECT *
       ,extract(epoch from mytimestamp)
       - lag(extract(epoch from mytimestamp))
       over (PARTITION BY user_id order by mytimestamp) as time_interval
FROM toy_data_psql;

# flag each session of given user, assuming a new session after 30min.
# Create a boolean column (1 indicating a new session, 0 otherwise).
SELECT *
  , CASE
      WHEN EXTRACT(EPOCH FROM mytimestamp)
           - LAG(EXTRACT(EPOCH FROM mytimestamp))
           OVER (PARTITION BY user_id ORDER BY mytimestamp) >= 30 * 60
      THEN 1
      ELSE 0
    END as new_session
FROM
  toy_data_psql;

#Create a cumulative sum over this boolean column to create a "session id". To make it easier to visualize, we can concatenate it with the userid, and then build our final "sessionid” column.
SELECT *
  , user_id || '_' || SUM(new_session)
  OVER (PARTITION BY user_id ORDER BY mytimestamp) AS session_id
FROM (
  SELECT *
    , CASE
       WHEN EXTRACT(EPOCH FROM mytimestamp)
          - LAG(EXTRACT(EPOCH FROM mytimestamp))
            OVER (PARTITION BY user_id ORDER BY mytimestamp) >= 30 * 60
       THEN 1
       ELSE 0
      END as new_session
    FROM
      toy_data_psql
) s1

# Python

In [2]:
import pandas as pd
from datetime import timedelta

In [79]:
# load dataset
toy_data = pd.read_csv('../Twitter/toy_data.csv',delimiter=';')

toy_data.head(5)

,user_id,mytimestamp
0,uid1,2013-09-04T15:49:49
1,uid1,2013-09-04T15:49:58
2,uid1,2013-09-04T16:37:11
3,uid1,2013-09-04T16:37:18
4,uid1,2013-09-04T16:39:27


In [80]:
# define treshold value
T = timedelta(seconds=30*60)
T

datetime.timedelta(0, 1800)

In [81]:
toy_data =  pd.concat([toy_data,
                       toy_data.groupby('user_id').transform(lambda x:x.shift(1))]
                      ,axis=1)

In [82]:
toy_data.columns = ['user_id','mytimestamp','prev_mytimestamp']

In [83]:
toy_data.dtypes

user_id             object
mytimestamp         object
prev_mytimestamp    object
dtype: object

In [84]:
# create the new session column
date_format = "%Y-%m-%dT%H:%M:%S"
d1 =  pd.to_datetime(toy_data['mytimestamp'], format= date_format)
d0 =  pd.to_datetime(toy_data['prev_mytimestamp'], format= date_format)

toy_data['difference'] = (d1 - d0)

In [85]:
toy_data['difference']

0         NaT
1    00:00:09
2    00:47:13
3    00:00:07
4    00:02:09
5    00:04:30
6    03:28:06
7    03:48:14
8    00:20:18
9         NaT
10   00:01:33
11   00:55:19
12   00:59:04
13   00:00:24
14   00:00:14
15   01:13:25
Name: difference, dtype: timedelta64[ns]

In [96]:
toy_data['new_session'] = (toy_data.difference >= T).astype(int)
toy_data.head()

,user_id,mytimestamp,prev_mytimestamp,difference,new_session
0,uid1,2013-09-04T15:49:49,NaN,NaT,0
1,uid1,2013-09-04T15:49:58,2013-09-04T15:49:49,00:00:09,0
2,uid1,2013-09-04T16:37:11,2013-09-04T15:49:58,00:47:13,1
3,uid1,2013-09-04T16:37:18,2013-09-04T16:37:11,00:00:07,0
4,uid1,2013-09-04T16:39:27,2013-09-04T16:37:18,00:02:09,0


In [97]:
# create the session_id
toy_data['increment'] = toy_data.groupby("user_id")['new_session'].cumsum()
toy_data['session_id'] = toy_data['user_id'].astype(str) + '_'  + toy_data['increment'].astype(str)

# to get the same result as with hive/postgresql
toy_data = toy_data.sort(['user_id','mytimestamp'])


/Users/iglopezfrancos/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [98]:
toy_data

,user_id,mytimestamp,prev_mytimestamp,difference,new_session,increment,session_id
0,uid1,2013-09-04T15:49:49,NaN,NaT,0,0,uid1_0
1,uid1,2013-09-04T15:49:58,2013-09-04T15:49:49,00:00:09,0,0,uid1_0
2,uid1,2013-09-04T16:37:11,2013-09-04T15:49:58,00:47:13,1,1,uid1_1
3,uid1,2013-09-04T16:37:18,2013-09-04T16:37:11,00:00:07,0,1,uid1_1
4,uid1,2013-09-04T16:39:27,2013-09-04T16:37:18,00:02:09,0,1,uid1_1
5,uid1,2013-09-04T16:43:57,2013-09-04T16:39:27,00:04:30,0,1,uid1_1
6,uid1,2013-09-04T20:12:03,2013-09-04T16:43:57,03:28:06,1,2,uid1_2
7,uid1,2013-09-05T00:00:17,2013-09-04T20:12:03,03:48:14,1,3,uid1_3
8,uid1,2013-09-05T00:20:35,2013-09-05T00:00:17,00:20:18,0,3,uid1_3
12,uid1,2013-09-05T01:19:39,2013-09-05T00:20:35,00:59:04,1,4,uid1_4
